# Monte Carlo Counterfactual regret minimization
Monte Carlo Counterfactual regret minimization ( **MCCFR** ) is a iterative algorithm that gradually converges to a nash equilibruim in a two player zero-sum game.
In this laboratory we will many deal wih Counterfactual regret minimization and not use the monte carlo part of these game. 
Due to game not being complex and large enough when traversing the possbile simulation of the match unfolding. 
To begin with lest cover some basic background to understand the MCCFR algroithm and the strategy it will predict.

## Nash equilibrium
Nash equilibrium is a concept in game theory where the optimal outcome of a game is one where on player has an incentive to deviate from the cosen strategy after considering an opponent's choice.
Overall, an individual can receive no incremental benefit from changing actions, assuming opposing players remain constant in their strategies.
A game may have multiple Nash equilibria or none at all.

If you want to test for a nash equilibrium simply reveal each person's strategy to all players.
The Nash equilibrium exists if no players change their strategy despite knowing the action of their opponents.
To note is that a nash equilibrium strategy does not mean a winning strategy but a strategy where no one losses by a large margin.

In the case of Rock, Paper and Scissors, Kuhn Poker or Poker Texas hold'em their exists a nash equilibrium strategy.


## Counterfactual regret minimization Algorithm
Counterfactual regret minimization ( **CFR** ) is a scenario based algorithm where each
possbile step is estimated with a regret sum (of previous state) after all possbile action have been taken in the descion tree.
Traversing upwards give regrets the knowledge of not have taking a certain action at each step.
CFR is counterfactual by assuming the action of the opponent and all of its possible action to take.
By minimizaing this regret the recommend playout strategy will appear. 
This strategy is is the goal of CFR algortihm. 
To find the strategy distribution is the optimal restult of all given actions and higher probability the better option it is.

* A simple breakdown of this algorithm[1] is:
    - Counterfactual: "if I had known"
    - Regret: "how much better would i have done if I did something else instead?"
    - Minimization: "What strategy minimizes my overall regret?"

### Monte Carlo
Monte Carlo a method of estimating the value of an unknown quatity using the principles of inferential statistics.
In the counterfactual regret minimization this means instead of a population of actions it is now a sample of action that meet some conditions of being worth traversing down the branch.
The monte carlo then drastically reduces any game with high combinations of simulations to a manageable computation.
The conditions of computing a simulation are parameters that ca be fine tune to a the game being played and if a action is consider "stupid".

### Rock, Paper, Scissors
In Rock Paper Scissors and every two-player zero-sum game: when both players use regret-matching to update their strategies,
the pair of average strategies converges to a Nash equilibrium as the number of iterations tends to infinity. 
At each iteration, both players update their regrets as above and then both each player computes their own new strategy based on their own regret tables.
Modify the RPSTrainer program above so that both players use regret matching. 
Compute and print the resulting unique equilibrium strategy.

In [1]:
import os
import numpy as np
import random
import matplotlib.pyplot as plt

In [40]:
# RPS = Rock, Paper, Scissors
class RPSTrainer:
    ROCK = 0
    PAPER = 1
    SCISSORS = 2
    NUM_ACTIONS = 3
    HAND = {0: 'Rock', 1: 'Paper', 2: 'Scissors'}

    def __init__(self, opponentStrategy):
        self.regretSum = np.zeros(self.NUM_ACTIONS, dtype=np.float64)
        self.strategySum = np.zeros(self.NUM_ACTIONS, dtype=np.float64)
        self.strategy = np.zeros(self.NUM_ACTIONS, dtype=np.float64)
        self.opponentStrategy = opponentStrategy
        self.opponentRealStrategy = np.zeros(self.NUM_ACTIONS, dtype=np.float64)

    # Get current mixed strategy through regret-matching
    def getStrategy(self):
        normalizingSum = 0
        for i in range(self.NUM_ACTIONS):
            self.strategy[i] = self.regretSum[i] if self.regretSum[i] > 0 else 0
            normalizingSum += self.strategy[i]

        for i in range(self.NUM_ACTIONS):
            self.strategy[i] = self.strategy[i] / normalizingSum if normalizingSum > 0 else 1.0 / self.NUM_ACTIONS
            self.strategySum[i] += self.strategy[i]
        
        return self.strategy
    
    def getAverageStrategy(self):
        avgStrategy = np.zeros(self.NUM_ACTIONS)
        normalizingSum = np.sum(self.strategySum)
        
        for i in range(self.NUM_ACTIONS):
            avgStrategy[i] = round(self.strategySum[i] / normalizingSum, 4) if normalizingSum > 0 else round(1.0 / self.NUM_ACTIONS, 4)
        return avgStrategy

    # Get random action according to mixed-strategy distribution
    def getAction(self, strategy):
        rr = random.random()
        cumlativeProbability = 0
        action = 0

        while action < self.NUM_ACTIONS-1:
            cumlativeProbability += strategy[action]
            if rr < cumlativeProbability:
                break
            action += 1

        return action


    # Train
    def train(self, iterations):
        actionUtility = np.zeros(self.NUM_ACTIONS)
        for _ in range(iterations):
            # Regret Mixed-strategy actions
            strategy = self.getStrategy()
            myAction = self.getAction(strategy)
            opponentAction = self.getAction(self.opponentStrategy)
            
            # Compute Action utilities 
            actionUtility[opponentAction] = 0
            actionUtility[(opponentAction + 1) % self.NUM_ACTIONS] = 1
            actionUtility[(opponentAction - 1) % self.NUM_ACTIONS] = -1
            
            # Accumulate action regrets
            for i in range(self.NUM_ACTIONS):
                self.regretSum[i] += actionUtility[i] - actionUtility[myAction]

    @classmethod
    def winner(cls, p1, p2):
        if p1 == p2:
            return 0
        elif (p1==cls.ROCK and p2==cls.SCISSORS) or (p1==cls.SCISSORS and p2==cls.PAPER) or (p1==cls.PAPER and p2==cls.ROCK):
            return 1
        else:
            return -1
        
    def play(self, iterations):
        myStrategy = self.getAverageStrategy()
        winCount, drawCount, lossCount = 0, 0, 0

        for _ in range(iterations):
            myAction = self.getAction(myStrategy)
            oppentAction = self.getAction(self.opponentStrategy)
            
            winner = self.winner(myAction, oppentAction)
            if winner == 1:
                winCount += 1
            elif winner == -1:
                 lossCount += 1
            else:
                drawCount += 1

        return (winCount, drawCount, lossCount)

In [15]:
print(10*"=", "Strategies", "="*10)
# Opponent probability distribution of choosing hand. This is also know as strategy here.
oppStrategy = np.array([0.0, 0.9, 0.1]) # 0 % Rock, 90% Paper, 10% Scissors
print(f"Opponent strategy: {oppStrategy}")
print("Expected strategy: [0, 0, 1]") # 100% Scissors.
# This Expected Strategy is because you can't loss by always playing Scissors

# Create it
engine_1 = RPSTrainer(oppStrategy)

# Train, it!
print(10*"=", "Training", "="*10)
iteration = 100000
engine_1.train(iteration)
print(f"{iteration} iterations trained")
print("-"*5 + ">" , f"Predicted strategy {engine_1.getAverageStrategy()}")

# Test, it!
print(10*"=", "Playing", "="*10)
print('Playing using my trained strategy:')
w, d, l = engine_1.play(1000)

print('win : ',w)
print('draw: ',d)
print('loss: ',l)

========== Strategies ==========
Opponent strategy: [0.  0.9 0.1]
Expected strategy: [0, 0, 1]
========== Training ==========
100000 iterations trained
-----> Predicted strategy [0. 0. 1.]
========== Playing ==========
Playing using my trained strategy:
win :  911
draw:  89
loss:  0


In [39]:
print(10*"=", "Strategies", "="*10)
# Opponent probability distribution of choosing hand. This is also know as strategy here.
# Nash equilibrium of 33 % Rock, 33% Paper, 33% Scissors
oppStrategy = np.array([1/3 for _ in range(3)])  # 33 % Rock, 33% Paper, 33% Scissors
print(f"Opponent strategy: {oppStrategy}")
print("Expected strategy: [1/3, 1/3, 1/3]") # 33 % Rock, 33% Paper, 33% Scissors

# Create it
engine = RPSTrainer(oppStrategy)

# Train, it!
print(10*"=", "Training", "="*10)
iteration = 1_000_000
engine.train(iteration)
print(f"{iteration} iterations trained")
print("-"*5 + ">" , f"Predicted strategy {engine.getAverageStrategy()}")

# Test, it!
print(10*"=", "Playing", "="*10)
print('Playing using my trained strategy:')
w, d, l = engine.play(1000)

print('win : ',w)
print('draw: ',d)
print('loss: ',l)

========== Strategies ==========
Opponent strategy: [0.33333333 0.33333333 0.33333333]
Expected strategy: [1/3, 1/3, 1/3]
========== Training ==========
1000000 iterations trained
-----> Predicted strategy [0.4475 0.1084 0.4441]
========== Playing ==========
Playing using my trained strategy:
win :  323
draw:  351
loss:  326


This Expected Strategy is because the best stategy for playing against an opponent following nash equilibrium of a game.
The best strategy to compare is the same strategy and since rock, paper, scissors nash equilibrium of the strategy in this game is 1/3 for every hand.

The predicted strategy clearly shows that it has not yet learned that be best strategy is playing 1/3. 
For that reason and CFR being a exploitative algorithm it is trying to find a strategy to miminze regret but the best strategy will have in a perfect world the same mounts of wins as losses or draws.

In [33]:
print(10*"=", "Strategies", "="*10)
# Opponent probability distribution of choosing hand. This is also know as strategy here.
oppStrategy = np.array([0.1, 0.8, 0.1]) # 10 % Rock, 80% Paper, 10% Scissors
print(f"Opponent strategy: {oppStrategy}")
print("Expected strategy: [0, 0, 1]") # 100% Scissors

# Create it
engine_2 = RPSTrainer(oppStrategy)

# Train, it!
print(10*"=", "Training", "="*10)
iteration = 100000
engine_2.train(iteration)
print(f"{iteration} iterations trained")
print("-"*3 + ">" , f"Predicted strategy {engine_2.getAverageStrategy()}")

# Test, it!
print(10*"=", "Playing", "="*10)
print('Playing using my trained strategy:')
w, d, l = engine_2.play(1000)

print('win : ',w)
print('draw: ',d)
print('loss: ',l)

========== Strategies ==========
Opponent strategy: [0.1 0.8 0.1]
Expected strategy: [0, 0, 1]
========== Training ==========
100000 iterations trained
---> Predicted strategy [0. 0. 1.]
========== Playing ==========
Playing using my trained strategy:
win :  806
draw:  95
loss:  99


This Expected Strategy is because to mimimize loss by always playing Scissors.
Eventually the opponent will throw a rock and then loss algorithm will loss a match.
But the highest probable hand played from opponent is paper and therefore the safest strategy is to always play scissors.

### Kuhn Poker
Kuhn poker is an extremely simplified form of poker.
Kuhn as a simple model zero-sum two-player imperfect-information game. 
In Kuhn poker, the deck includes only three playing cards, in this deck there will be a Ace, King and Queen. 

* Play:
    - One card is dealt to each player, which may place bets similarly to a standard poker. Both player have now the option to either bet or pass.
    - If both players bet or both players pass, the player with the higher card wins, otherwise, the betting player wins.

In [2]:
from kuhn import Kuhn

In [3]:
# Setup
kuhn = Kuhn()
kuhn.new_match("human","bot")

# First round / Pre showdown
hands = kuhn.pre_showdown()

# Our hand
print("Our human card", hands["human"])

# Showdown
winner = kuhn.showdown()
print(f"\nHands human: {hands['human']} and bot: {hands['bot']}", "\nWinner is:", winner)

Our human card J

Hands human: J and bot: Q 
Winner is: bot


In [183]:
class Node:
    PASS = 0
    BET = 1
    NUM_ACTIONS = 2
    
    def __init__(self):
        # Kuhn node definitions
        self.infoSet = ""        
        self.regretSum = np.zeros(self.NUM_ACTIONS, dtype=np.float64)
        self.strategy = np.zeros(self.NUM_ACTIONS, dtype=np.float64)
        self.strategySum = np.zeros(self.NUM_ACTIONS, dtype=np.float64)
    
    # Get current information set mixed strategy through regret-matching
    def getStrategy(self, realizationWeight):
        normalizingSum = 0
        for i in range(self.NUM_ACTIONS):
            self.strategy[i] = self.regretSum[i] if self.regretSum[i] > 0 else 0
            normalizingSum += self.strategy[i]

        for i in range(self.NUM_ACTIONS):
            self.strategy[i] = self.strategy[i] / normalizingSum if normalizingSum > 0 else 1.0 / self.NUM_ACTIONS
            self.strategySum[i] += realizationWeight * self.strategy[i]
        
        return self.strategy
    
    # Get average infrmation set mixed strategy across all training iterations
    def getAverageStrategy(self):
        avgStrategy = np.zeros(self.NUM_ACTIONS)
        normalizingSum = np.sum(self.strategySum)
        
        for i in range(self.NUM_ACTIONS):
            avgStrategy[i] = round(self.strategySum[i] / normalizingSum, 4) if normalizingSum > 0 else round(1.0 / self.NUM_ACTIONS, 4)
        return avgStrategy

    # Get information set string representation
    def __str__(self):
        return f"{self.infoSet}: {self.getAverageStrategy()}"

In [179]:
class KuhnTrainer:
    # Kuhn poker definitions
    PASS = 0
    BET = 1
    NUM_ACTIONS = 2
    FOLD = "f" # f for FOLD
    CALL = "c" # c for CALL
    DOUBLE_CALL = "cc"
    
    def __init__(self):
        self.nodeMap = {}
    
    def cfr(self, cards, history, p0, p1):
        plays = len(history)
        player = plays % 2
        opponent = 1 - player
        # Return payoff for terminal states
        if plays > 1:
            terminalPass = history[plays -1] == self.FOLD
            doubleBet = history[plays - 2: plays] == self.DOUBLE_CALL
            isPlayerCardHigher = cards[player] > cards[opponent]
            if terminalPass:
                if history == self.DOUBLE_CALL:
                    return 1 if isPlayerCardHigher else -1
                return 1
            elif doubleBet:
                return 2 if isPlayerCardHigher else -2

        infoSet = f"{cards[player]}" + history

        # Get information set node or create it if non-existant
        node = self.nodeMap.get(infoSet, None)
        if node is None:
            node = Node()
            node.infoSet = infoSet
            self.nodeMap[infoSet] = node

        # for each action, recursively call cfr with additional history and probability
        strategy = node.getStrategy(p0 if player == 0 else p1)
        util = np.zeros(self.NUM_ACTIONS)
        nodeUtil = 0
        for a in range(self.NUM_ACTIONS):
            nextHistory = history + (self.FOLD if a == 0 else self.CALL)
            util[a] = - self.cfr(cards, nextHistory, p0 * strategy[a], p1) if player == 0 else - self.cfr(cards, nextHistory, p0, p1 * strategy[a])
            nodeUtil += strategy[a] * util[a]

        # for each action, compute the accumulate couterfactual regret
        for a in range(self.NUM_ACTIONS):
            regret = util[a] - nodeUtil
            node.regretSum[a] += (p1 if player == 0 else p0) * regret

        return nodeUtil
    
    def train(self, iterations):
        util = 0

        for i in range(iterations):
            # shuffle cards
            arr = [1, 2, 3]
            random.shuffle(arr)
            CARDS = { k:v for k,v in enumerate(arr) }

            util += self.cfr(CARDS, "", 1, 1)

        print("Average game value: ", util / iterations)
        for node in self.nodeMap:
            state = str(node)
            print(f"{Kuhn.value_deck[int(state[0])]} {state[1:]}")
            
    def play(self, iterations):
        winCount, drawCount, lossCount = 0, 0, 0

        for _ in range(iterations):
            winner = 0
            kuhn.new_match("human","bot")
            hands = kuhn.pre_showdown()
            
            myStrategy = self.nodeMap.get(f"{Kuhn.deck_value[hands['bot']]}{self.CALL}", None)
            myAction = myStrategy.getAverageStrategy() if myStrategy is not None else [1., 0.]
            if np.argmax(myAction) == 1:
                winner = kuhn.showdown()
                winner = 1 if winner == "bot" else -1

            if winner == 1:
                winCount += 1
            elif winner == -1:
                 lossCount += 1
            else:
                drawCount += 1

        return (winCount, drawCount, lossCount)

In [184]:
kuhn_master = KuhnTrainer()

print(10*"=", "Training", "="*10)
iterations = 1_000_000
kuhn_master.train(iterations)
print(f"{iterations} iterations trained")

========== Training ==========
Average game value:  0.11119579603942997
K 
J f
K fc
J c
Q 
K f
Q fc
K c
Q f
Q c
J 
J fc
1000000 iterations trained


In [185]:
print(10*"=", "Playing", "="*10)
print('Playing using Counterfactual Regret Minimization trained strategy:')
w, d, l = kuhn_master.play(1000)
print('win : ',w)
print('draw: ',d)
print('loss: ',l)

========== Playing ==========
Playing using Counterfactual Regret Minimization trained strategy:
win :  482
draw:  349
loss:  169


# Resources:
- Paper [An Introduction to Counterfactual Regret Minimization](http://modelai.gettysburg.edu/2013/cfr/cfr.pdf)
- Tech Talk: [Libratus & Conterfactual Regret Minimization](https://www.youtube.com/watch?v=htRtfyab-Ns) **[1]**